In this exercise, you will leverage what you've learned in the two previous tutorials to deal with **categorical variables** and **missing values** in a challenging dataset.

# Setup

The questions below will give you feedback on your work. Run the following cell to set up the feedback system.

In [ ]:
# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.ml_level_2_dev.ex2 import *
print("Setup Complete")

You will work with the [Ames Housing dataset](https://www.kaggle.com/c/house-prices-advanced-regression-techniques). 

![Ames Housing dataset image](./images/ex1_housesbanner.png)

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, your goal is to predict the final price of each home.  

Run the next code cell without changes to load the training and validation sets in `X_train`, `X_valid`, `y_train`, and `y_valid`.  The test set is loaded in `X_test`.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# read the data
train_data = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv', index_col='Id')
test_data = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv', index_col='Id')

# remove rows with missing target, separate target from predictors
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = train_data.SalePrice
train_data.drop(['SalePrice'], axis=1, inplace=True)

# "cardinality" means the number of unique values in a column
# select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in train_data.columns if
                        train_data[cname].nunique() < 10 and
                        train_data[cname].dtype == "object"]

# select numeric columns
numeric_cols = [cname for cname in train_data.columns if
                train_data[cname].dtype in ['int64', 'float64']]

# keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X = train_data[my_cols]
X_test = test_data[my_cols]

# break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

Use the next code cell to print the first several rows of the data.

In [ ]:
X_train.head()

You'll use the same `score_dataset()` function from the previous tutorial.  This function reports the [mean absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error) (MAE) from a random forest model.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

this is what you're gonna do.

![Ames Housing dataset image](./images/ex2_steps.png)

# Step 1: Deal with categorical variables

In [ ]:
X_train_1 = ____
X_valid_1 = ____

step_1.check()

In [ ]:
# one-hot encode categorical data
X_train_1 = pd.get_dummies(X_train)
X_valid_1 = pd.get_dummies(X_valid)

# ensure columns are in same order in both datasets
X_train_1, X_valid_1 = X_train_1.align(X_valid_1, join='inner', axis=1)

step_1.assert_check_passed()

In [ ]:
step_1.hint()
step_1.solution()

# Step 2: Deal with missing values

In [ ]:
X_train_2 = ____
X_valid_2 = ____

step_2.check()

In [ ]:
from sklearn.impute import SimpleImputer

# make copy to avoid changing original data (when imputing)
X_train_imp = X_train_1.copy()
X_valid_imp = X_valid_1.copy()

# get names of columns with missing values
X_1 = pd.concat([X_train_1, X_valid_1])
cols_with_missing = [col for col in X_1.columns if X_1[col].isnull().any()]

# make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_imp[col + '_was_missing'] = X_train_1[col].isnull()
    X_valid_imp[col + '_was_missing'] = X_valid_1[col].isnull()
    
# imputation
my_imputer = SimpleImputer()
X_train_2 = pd.DataFrame(my_imputer.fit_transform(X_train_imp))
X_valid_2 = pd.DataFrame(my_imputer.transform(X_valid_imp))

# imputation removed column names; put them back
X_train_2.columns = X_train_imp.columns
X_valid_2.columns = X_valid_imp.columns

step_2.assert_check_passed()

In [ ]:
step_2.hint()
step_2.solution()

# Step 3: Test your performance

In [ ]:
score = score_dataset(X_train_2, X_valid_2, y_train, y_valid)
print('MAE score:', score)

step_3.check()

In [ ]:
score = score_dataset(X_train_2, X_valid_2, y_train, y_valid)
print('MAE score:', score)

step_3.assert_check_passed()

In [ ]:
step_3.hint()

# Keep going

Move on to learn all about how to use **[pipelines](#$NEXT_NOTEBOOK_URL$)** to improve your machine learning code!